In [1]:
import numpy as np
import itertools
import warnings
import matplotlib.pyplot as plt

In [2]:
import tenpy

from tenpy.networks.site import Site, SpinHalfFermionSite, SpinHalfSite, GroupedSite, SpinSite
from tenpy.tools.misc import to_iterable, to_iterable_of_len, inverse_permutation
from tenpy.networks.mps import MPS  # only to check boundary conditions

from tenpy.models.lattice import Lattice, _parse_sites
from tenpy.models.model import CouplingMPOModel, NearestNeighborModel
from tenpy.tools.params import get_parameter

from tenpy.algorithms import dmrg

In [3]:
# statistics
from scipy.optimize import curve_fit
from scipy import stats

In [28]:
# some api for the file operation
import h5py
from tenpy.tools import hdf5_io
import os.path
# data path
prefix = 'data/'

In [4]:
__all__ = ['KitaevLadder', 'KitaevLadderModel']

In [6]:
class KitaevLadder(Lattice):    
    """ A ladder coupling two chains of the Kitaev form
    .. image :: /images/lattices/Ladder.*
    Parameters
    ----------
    L : int
        The length of each chain, we have 2*L sites in total.
    sites : (list of) :class:`~tenpy.networks.site.Site`
        The two local lattice sites making the `unit_cell` of the :class:`Lattice`.
        If only a single :class:`~tenpy.networks.site.Site` is given, it is used for both chains.
    **kwargs :
        Additional keyword arguments given to the :class:`Lattice`.
        `basis`, `pos` and `pairs` are set accordingly.
    """
    dim = 1

    def __init__(self, L, sites, **kwargs):
        sites = _parse_sites(sites, 4)
        basis = np.array([[2., 0.]])
        pos = np.array([[0., 0.], [0., 1.], [1., 0.], [1., 1.]])
        kwargs.setdefault('basis', basis)
        kwargs.setdefault('positions', pos)
        kwargs.setdefault('bc', 'periodic')
        kwargs.setdefault('bc_MPS', 'infinite')
        
        NNz = [(0, 1, np.array([0])), (2, 3, np.array([0]))]
        NNx = [(1, 3, np.array([0])), (2, 0, np.array([1]))]
        NNy = [(0, 2, np.array([0])), (3, 1, np.array([1]))]
        nNNa = [(1, 2, np.array([0])), (3, 0, np.array([1]))]
        nNNb = [(0, 3, np.array([0])), (2, 1, np.array([1]))]
        
        kwargs.setdefault('pairs', {})
        kwargs['pairs'].setdefault('nearest_neighbors_x', NNx)
        kwargs['pairs'].setdefault('nearest_neighbors_y', NNy)
        kwargs['pairs'].setdefault('nearest_neighbors_z', NNz)
        kwargs['pairs'].setdefault('next_nearest_neighbors_a', nNNa)
        kwargs['pairs'].setdefault('next_nearest_neighbors_b', nNNb)
        
        Lattice.__init__(self, [L], sites, **kwargs)
        
        
class KitaevLadderModel(CouplingMPOModel):
    def __init__(self, model_params):
        CouplingMPOModel.__init__(self, model_params)

    def init_sites(self, model_params):
        conserve = get_parameter(model_params, 'conserve', None, self.name)
        fs = SpinHalfSite(conserve=conserve)
        return [fs, fs, fs, fs]

    def init_lattice(self, model_params):
        L = get_parameter(model_params, 'L', 3, self.name)
        gs = self.init_sites(model_params)
        model_params.pop("L")
        lat = KitaevLadder(L, gs)
        return lat

    def init_terms(self, model_params):
        Jx = get_parameter(model_params, 'Jx', 1., self.name, True)
        Jy = get_parameter(model_params, 'Jy', 1., self.name, True)
        Jz = get_parameter(model_params, 'Jz', 1., self.name, True)

        for u1, u2, dx in self.lat.pairs['nearest_neighbors_z']:
            self.add_coupling(-Jz, u1, 'Sx', u2, 'Sx', dx)
        for u1, u2, dx in self.lat.pairs['nearest_neighbors_x']:
            self.add_coupling(-Jx, u1, 'Sz', u2, 'Sz', dx)
        for u1, u2, dx in self.lat.pairs['nearest_neighbors_y']:
            self.add_coupling(-Jy, u1, 'Sy', u2, 'Sy', dx)
         
        
def plot_lattice():
    fig, ax = plt.subplots()
    lat = KitaevLadder(5, None, bc='periodic')
    links_name = 'nearest_neighbors_z'
    lat.plot_coupling(ax, lat.pairs[links_name], linewidth=5.)
    # print(lat.pairs['nearest_neighbors'])
    print(lat.unit_cell)
    lat.plot_order(ax=ax, linestyle='--')
    lat.plot_sites(ax)
    # lat.plot_basis(ax, color='g', linewidth=3.)
    ax.set_aspect('equal')
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    ax.axis('off')
    # plt.title(links_name)
    plt.show()


## Now we define the function for calculating the ground energy and ground wave functions when the bond dimensions are given with specified Hamiltonian parameters

In [7]:
def run_atomic(
    chi=30,
    Jx=1., 
    Jy=1., 
    Jz=0., 
    L=1, 
    verbose=1, 
    calc_correlation=True,
):

    #######################
    # set the paramters for model initialization
    model_params = dict(conserve=None, Jx=Jx, Jy=Jy, Jz=Jz, L=L, verbose=verbose)
    # providing a product state as the initial state
    prod_state = ["up", "up"] * (2 * model_params['L'])
    # initialize the model
    M = KitaevLadderModel(model_params)
    psi = MPS.from_product_state(
        M.lat.mps_sites(), 
        prod_state, 
        bc=M.lat.bc_MPS,
    )
    #######################

    
    #######################
    # set the parameters for the dmrg routine
    dmrg_params = {
        'start_env': 10,
#         'mixer': False,  # setting this to True helps to escape local minima
        'mixer': True,
        'mixer_params': {
            'amplitude': 1.e-5,
            'decay': 1.2,
            'disable_after': 30
        },
        'trunc_params': {
            'chi_max': 4,
            'svd_min': 1.e-10,
        },
        'max_E_err': 1.e-6,
        'max_S_err': 1.e-4,
        'max_sweeps': 1000,
        'verbose': verbose,
    }
    #######################
    
    if verbose:
        print("\n")
        print("=" * 80)
        print("="*30 + "START" + "="*30)
        print("=" * 80)
        print("Chi = ", chi, '\n')

    # here we create another new engine for every new chi
    # this is quite unusual but since some strange issues
    # we can only treat like this
    eng = dmrg.TwoSiteDMRGEngine(psi, M, dmrg_params)
    eng.reset_stats()
    eng.trunc_params['chi_max'] = chi
    info = eng.run()
#         print("INFO: \n", info)

    if verbose:
        print("Before the canonicalization:")
        print("Bond dim = ", psi.chi)

        print("Canonicalizing...")
        psi_before = psi.copy()

    psi.canonical_form()

    if verbose:
        ov = psi.overlap(psi_before, charge_sector=0)
        print("The norm is: ",psi.norm)
        print("The overlap is: ", ov)
        print("After the canonicalization:")
        print("Bond dim = ", psi.chi)

        print("Computing properties")

    energy=info[0]

    if verbose:
        print("Optimizing")

    tenpy.tools.optimization.optimize(3)

    if verbose:
        print("Loop for chi=%d done." % chi)
        print("=" * 80)
        print("="*30 + " END " + "="*30)
        print("=" * 80)
        
    return (energy, psi.copy())


In [8]:
def run(
    chi_list = [int(2**i) for i in np.arange(4, 7, 1)],
    Jx=1., 
    Jy=1., 
    Jz=0., 
    L=1, 
    verbose=1, 
    calc_correlation=True,
):

    #######################
    # set the paramters for model initialization
    model_params = dict(conserve=None, Jx=Jx, Jy=Jy, Jz=Jz, L=L, verbose=verbose)
    # providing a product state as the initial state
    prod_state = ["up", "up"] * (2 * model_params['L'])
    # initialize the model
    M = KitaevLadderModel(model_params)
    psi = MPS.from_product_state(
        M.lat.mps_sites(), 
        prod_state, 
        bc=M.lat.bc_MPS,
    )
    #######################

    
    #######################
    # set the parameters for the dmrg routine
    dmrg_params = {
        'start_env': 10,
#         'mixer': False,  # setting this to True helps to escape local minima
        'mixer': True,
        'mixer_params': {
            'amplitude': 1.e-5,
            'decay': 1.2,
            'disable_after': 30
        },
        'trunc_params': {
            'chi_max': 4,
            'svd_min': 1.e-10,
        },
        'max_E_err': 1.e-10,
        'max_S_err': 1.e-8,
        'max_sweeps': 1000,
        'verbose': verbose,
    }
    #######################
    
    
    # we will return this data dictionary in the end, for the further investigation
    data = dict(
        chi_list=[],
        energy_list=[], 
        psi_list=[],
#         entropy_list=[], 
#         correlation_list=[],
    )
    
#     chi_list = np.arange(4, 129)
#     chi_list = [int(2**i) for i in np.arange(4, 7, 0.5)]
    data['chi_list'] = chi_list
    for chi in chi_list:
        
        if verbose:
            print("\n")
            print("=" * 80)
            print("="*30 + "START" + "="*30)
            print("=" * 80)
            print("Chi = ", chi, '\n')
        
        # here we create another new engine for every new chi
        # this is quite unusual but since some strange issues
        # we can only treat like this
        eng = dmrg.TwoSiteDMRGEngine(psi, M, dmrg_params)
        eng.reset_stats()
        eng.trunc_params['chi_max'] = chi
        info = eng.run()
#         print("INFO: \n", info)

        if verbose:
            print("Before the canonicalization:")
            print("Bond dim = ", psi.chi)

            print("Canonicalizing...")
            psi_before = psi.copy()
            
        psi.canonical_form()
        
        if verbose:
            ov = psi.overlap(psi_before, charge_sector=0)
            print("The norm is: ",psi.norm)
            print("The overlap is: ", ov)
            print("After the canonicalization:")
            print("Bond dim = ", psi.chi)

            print("Computing properties")
        
        energy=info[0]
        data['energy_list'].append(energy)
        data['psi_list'].append(psi.copy())
        
#         entropy = np.mean(psi.entanglement_entropy())
#         data['entropy_list'].append(entropy)
        
#         if calc_correlation:
#             correlation = psi.correlation_length()
#             data['correlation_list'].append(correlation)
        
        if verbose:
            print("Optimizing")
            
        tenpy.tools.optimization.optimize(3)
        
        if verbose:
            print("Loop for chi=%d done." % chi)
            print("=" * 80)
            print("="*30 + " END " + "="*30)
            print("=" * 80)
            
        pass # end of the for loop
        
    return data


In [9]:
def run_more(
    chi_list = [int(2**i) for i in np.arange(4, 7, 1)],
    Jx_list=[1.], 
    Jy_list=[1.], 
    Jz_list=[0.], 
    L=1, 
    verbose=1, 
):
    data_set = [[[run(chi_list=chi_list, Jx=Jx, Jy=Jy, Jz=Jz, L=L, verbose=verbose,) for Jx in Jx_list] for Jy in Jy_list] for Jz in Jz_list]
    return data_set

In [10]:
def plot_results(data):

    chi_list = data['chi_list']
    energy_list = data['energy_list']
    entropy_list = data['entropy_list']
    correlation_list = data['correlation_list']
    
    plt.plot(chi_list, energy_list, '-o')
    plt.xlabel(r"Bond dimension $\chi$")
    plt.ylabel(r"Ground Energy $E$")
    plt.show()
    
    plt.plot(chi_list, entropy_list, '-o')
    plt.xlabel(r"Bond dimension $\chi$")
    plt.ylabel(r"Entanglement Entropy $S$")
    plt.show()
    
    plt.plot(chi_list, correlation_list, '-o')
    plt.xlabel(r"Bond dimension $\chi$")
    plt.ylabel(r"Correlation Length $\xi$")
    plt.show()
    
    plt.plot(np.log(correlation_list), entropy_list, '-o')
    plt.xlabel(r"log Correlation Length $\xi$")
    plt.ylabel(r"Entanglement Entropy $S$")
    plt.show()

In [42]:
def naming(chi, Jx, Jy, Jz, L):
    return "KitaevLadder"+"_chi_"+str(chi)+"_Jx_"+str(Jx)+"_Jy_"+str(Jy)+"_Jz_"+str(Jz)+"_L_"+str(L)

def full_path(chi, Jx, Jy, Jz, L):
    return prefix+naming(chi, Jx, Jy, Jz, L)+".h5"
""" run the atomic and then save it
"""
def run_save(
    chi=30,
    Jx=1., 
    Jy=1., 
    Jz=0., 
    L=1, 
    verbose=1, 
    calc_correlation=True,
):
    file_name = full_path(chi, Jx, Jy, Jz, L)
    
    # if the file already existed then don't do the computation again
    if os.path.isfile(file_name):
        print("This file already existed")
    else:
        (energy, psi) = run_atomic(
            chi=chi, 
            Jx=Jx, 
            Jy=Jy, 
            Jz=Jz, 
            L=L, 
            verbose=verbose, 
            calc_correlation=calc_correlation,
        )
        data = {
            "psi": psi,
            "energy": energy,
            "parameters": {
                "chi": chi,
                "Jx": Jx,
                "Jy": Jy,
                "Jz": Jz,
                "L": L,
            }
        }
        with h5py.File(file_name, 'w') as f:
            hdf5_io.save_to_hdf5(f, data)
    pass

def read_psi(
    chi=30,
    Jx=1., 
    Jy=1., 
    Jz=0., 
    L=1, 
):
    file_name = full_path(chi, Jx, Jy, Jz, L)
    with h5py.File(file_name, 'r') as f:
        data = hdf5_io.load_from_hdf5(f)
        # or for partial reading:
        psi = hdf5_io.load_from_hdf5(f, "/psi")
        return psi
    
def read_energy(
    chi=30,
    Jx=1., 
    Jy=1., 
    Jz=0., 
    L=1, 
):
    file_name = full_path(chi, Jx, Jy, Jz, L)
    with h5py.File(file_name, 'r') as f:
        data = hdf5_io.load_from_hdf5(f)
        # or for partial reading:
        energy = hdf5_io.load_from_hdf5(f, "/energy")
        return energy
    
def read_parameters(
    chi=30,
    Jx=1., 
    Jy=1., 
    Jz=0., 
    L=1, 
):
    file_name = full_path(chi, Jx, Jy, Jz, L)
    with h5py.File(file_name, 'r') as f:
        data = hdf5_io.load_from_hdf5(f)
        # or for partial reading:
        parameters = hdf5_io.load_from_hdf5(f, "/parameters")
        return parameters

In [46]:
run_save()
print(read_psi())

This file already existed
MPS, L=4, bc='infinite'.
chi: [30, 30, 30, 30]
sites: SpinHalfSite(None) SpinHalfSite(None) SpinHalfSite(None) SpinHalfSite(None)
forms: (0.0, 1.0) (0.0, 1.0) (0.0, 1.0) (0.0, 1.0)


## Run and get the data pack

Here we individually run the function and return the data.
No matter how the other parts are designed, this process needs to be seperate since it consumes the majority of resources.

In [10]:
# data = run(chi_list=range(30, 60, 2), verbose=0)

## Regional run and multiprocessing

In [11]:
import multiprocessing

In [12]:
# region selection
Jx_list = [1.]
Jy_list = np.arange(-2, 2, 1)
Jz_list = np.arange(-2, 2, 1)
chi_list = range(30, 60, 2)
J_list = [(Jx, Jy, Jz, chi) for Jx in Jx_list for Jy in Jy_list for Jz in Jz_list for chi in chi_list]

# prepare other arguments
verbose = 1

In [13]:
# here the 'point' refers to the point in the parameter space J=(J_x, J_y, J_z)
def run_at_single_point(J):
    assert (len(J) == 4)
    return run_atomic(chi=J[3], Jx=J[0], Jy=J[1], Jz=J[2], verbose=verbose)

In [14]:
# multiprocessing preparation
cores = multiprocessing.cpu_count()
pool = multiprocessing.Pool(processes=cores)

result = pool.map(run_at_single_point, J_list)

parameter 'verbose'=1 for KitaevLadderModel
parameter 'verbose'=1 for KitaevLadderModel
parameter 'verbose'=1 for KitaevLadderModel
parameter 'verbose'=1 for KitaevLadderModel
parameter 'verbose'=1 for KitaevLadderModel
parameter 'verbose'=1 for KitaevLadderModel
parameter 'verbose'=1 for KitaevLadderModel
parameter 'verbose'=1 for KitaevLadderModel
parameter 'verbose'=1 for KitaevLadderModel
parameter 'verbose'=1 for KitaevLadderModel
parameter 'L'=1 for KitaevLadderModel
parameter 'L'=1 for KitaevLadderModel
parameter 'L'=1 for KitaevLadderModel
parameter 'L'=1 for KitaevLadderModel
parameter 'L'=1 for KitaevLadderModel
parameter 'L'=1 for KitaevLadderModel
parameter 'L'=1 for KitaevLadderModel
parameter 'conserve'=None for KitaevLadderModel
parameter 'L'=1 for KitaevLadderModel
parameter 'L'=1 for KitaevLadderModel
parameter 'conserve'=None for KitaevLadderModel
parameter 'conserve'=None for KitaevLadderModel
parameter 'conserve'=None for KitaevLadderModel
parameter 'conserve'=None 

## Extract the data

Here we extract the data into corresponding containers (lists)
Also some necessary preprocessing will be done here

In [ ]:
psi_list = data['psi_list']
entropy_list = [np.mean(psi.entanglement_entropy()) for psi in psi_list]
correlation_list = [psi.correlation_length() for psi in psi_list]

## Statistics

Do the fitting

In [ ]:
log_chi = np.log(correlation_list)
S = entropy_list

def func(log_chi, c, a):
    return (c / 6) * log_chi + a

fitParams, fitCovariances = curve_fit(func, log_chi, S)

## Plot the data

In [ ]:
plt.plot(log_chi, S, 'o')
plt.plot(log_chi, func(log_chi, fitParams[0], fitParams[1]), label=(r'$ S = \frac{%.2f}{6} \log\chi + %.2f $' % (fitParams[0], fitParams[1])))
plt.xlabel(r'$\log\chi$')
plt.ylabel(r'$S$')
plt.legend()
plt.title('Finite Engtanglement Scaling')
plt.show()

## Finite Entanglement Scaling
The below content only works for the gapless cases

In [ ]:
# from scipy.optimize import curve_fit

# def func(xi, c, a):
#     return (c / 6) * np.log(xi) + a

In [ ]:
# xi = correlation_list
# log_xi = np.log(correlation_list)
# s = entropy_list
# fitParams, fitCovariances = curve_fit(func, xi, s)

In [ ]:
# fitCovariances